In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import chardet
import os
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

2024-08-09 06:52:13.758198: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 06:52:13.758693: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 06:52:13.760828: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 06:52:13.766764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 06:52:13.777062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:
# 데이터셋 경로
file_path = '/home/jys0714/바탕화면/-/datasets/McDonald_s_Reviews.csv'

# 파일 인코딩 확인 및 데이터 로드

df = pd.read_csv(file_path, encoding='latin1')
df = df.dropna()

# df = df.head(100)

# # 데이터셋의 첫 몇 행 출력
# print(df.head())

# # 데이터셋 정보 출력
# print(df.info())

# # 리뷰 컬럼의 샘플 데이터 출력
# print(df['review'].sample(10))



In [4]:
# 특수 문자를 제거하는 함수 정의
def remove_invalid_chars(text):
    return re.sub(r'[^\x00-\x7F]+', '', text)

# review 컬럼에서 특수 문자 제거
df['review'] = df['review'].apply(remove_invalid_chars)


# 필요한 열들만 선택
X = df[['store_address', 'latitude ', 'longitude', 'rating_count', 'review_time']]
review = df['review']
target = df['rating'].astype(str).str[0].astype(int)

In [5]:
def rate_categorize(rate):
    if rate > 3:
        return 1
    else:
        return 0

target = target.apply(rate_categorize)
target

0        0
1        1
2        0
3        1
4        0
        ..
33391    0
33392    1
33393    1
33394    1
33395    1
Name: rating, Length: 32736, dtype: int64

In [6]:
from sklearn.preprocessing import OneHotEncoder
# 원-핫 인코딩할 열들 선택
categorical_columns = ['store_address', 'latitude ', 'longitude', 'review_time']

# 원-핫 인코딩 수행
one_hot_encoder = OneHotEncoder(sparse_output =False)
one_hot_encoded = one_hot_encoder.fit_transform(df[categorical_columns])


In [7]:
import myprocess

padded_sequences, token_counts = myprocess.preprocessing_tokinize(review)


In [8]:
# 텐서로 변환
review_tensor = tf.convert_to_tensor(padded_sequences, dtype=tf.float64)
one_hot_tensor = tf.convert_to_tensor(one_hot_encoded, dtype=tf.float64)
target_tensor = tf.convert_to_tensor(target.values, dtype=tf.float64)
combined_features = tf.concat([review_tensor, one_hot_tensor], axis=1)

I0000 00:00:1723153941.416311  309369 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-09 06:52:21.416531: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
# TensorFlow 데이터셋으로 변환
combined_ds = tf.data.Dataset.from_tensor_slices((combined_features, target_tensor))

# 결합된 데이터셋의 샘플 확인
for features, label in combined_ds.take(3):
    tf.print(features[:10], label)

[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 1
[0 0 0 ... 0 0 0] 0


2024-08-09 06:52:22.961017: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
process = myprocess.PreProcess(combined_ds, target.shape[0], is_tokenize=True)
len(token_counts)

14095

In [11]:
batch_size = 32

train_data, valid_data, test_data = process.get_datas(batch_size, train_size=0.4, valid_size = 0.1, test_size=0.5)

In [12]:
one_hot_encoded.shape[1]

156

In [13]:
vocab_size = len(token_counts) + one_hot_encoded.shape[1] + 2  
node_nums = [32, 64]
embedding_dims = [75, 100, 125]
layer_nums = [1, 2]

model_list= myprocess.make_models(vocab_size, node_nums, embedding_dims, layer_nums)
model_list

[<Sequential name=sequential, built=False>,
 <Sequential name=sequential_1, built=False>,
 <Sequential name=sequential_2, built=False>,
 <Sequential name=sequential_3, built=False>,
 <Sequential name=sequential_4, built=False>,
 <Sequential name=sequential_5, built=False>,
 <Sequential name=sequential_6, built=False>,
 <Sequential name=sequential_7, built=False>,
 <Sequential name=sequential_8, built=False>,
 <Sequential name=sequential_9, built=False>,
 <Sequential name=sequential_10, built=False>,
 <Sequential name=sequential_11, built=False>]

In [13]:
myprocess.compile_train_save(model_list, 10, train_data, valid_data, test_data )

Epoch 1/10


512/512 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.5312 - loss: 0.6827 - val_accuracy: 0.6728 - val_loss: 0.6282
Epoch 2/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.6631 - loss: 0.6328 - val_accuracy: 0.6541 - val_loss: 0.6246
Epoch 3/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.6541 - loss: 0.6263 - val_accuracy: 0.6651 - val_loss: 0.6140
Epoch 4/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.6602 - loss: 0.6137 - val_accuracy: 0.7039 - val_loss: 0.5847
Epoch 5/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.6726 - loss: 0.6036 - val_accuracy: 0.5176 - val_loss: 0.6920
Epoch 6/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.5176 - loss: 0.6869 - val_accuracy: 0.5203 - val_loss: 0.6745
Epoch 7/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.5228 - loss: 0.6761 - val_accuracy: 0.8097 - val_loss: 0.4562
Epoch 8/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.8371 - loss: 0.3924 - val_accurac

410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9436 - loss: 0.1496
Epoch 1/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - accuracy: 0.5022 - loss: 0.6952 - val_accuracy: 0.5176 - val_loss: 0.6858
Epoch 2/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.5067 - loss: 0.6854 - val_accuracy: 0.5176 - val_loss: 0.6769
Epoch 3/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.5402 - loss: 0.6752 - val_accuracy: 0.5197 - val_loss: 0.6767
Epoch 4/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.5238 - loss: 0.6750 - val_accuracy: 0.6685 - val_loss: 0.6200
Epoch 5/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.6607 - loss: 0.6250 - val_accuracy: 0.6737 - val_loss: 0.6134
Epoch 6/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.6708 - loss: 0.6103 - val_accuracy: 0.7482 - val_loss: 0.5104
Epoch 7/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 36ms/step - accuracy: 0.7579 - loss: 0.5005 - val_accuracy: 0.8191 - val_loss: 0.4133
Epoch 8/10
512

410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8759 - loss: 0.3000
Epoch 1/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.5067 - loss: 0.6902 - val_accuracy: 0.5225 - val_loss: 0.6792
Epoch 2/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.6537 - loss: 0.5595 - val_accuracy: 0.8753 - val_loss: 0.2960
Epoch 3/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.8910 - loss: 0.2704 - val_accuracy: 0.9233 - val_loss: 0.2031
Epoch 4/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.9284 - loss: 0.1955 - val_accuracy: 0.9404 - val_loss: 0.1525
Epoch 5/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.9494 - loss: 0.1454 - val_accuracy: 0.9499 - val_loss: 0.1276
Epoch 6/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.9607 - loss: 0.1173 - val_accuracy: 0.9584 - val_loss: 0.1071
Epoch 7/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.9624 - loss: 0.1098 - val_accuracy: 0.9649 - val_loss: 0.1001
Epoch 8/10
512/

410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.9754 - loss: 0.0725
Epoch 1/10
288/512 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.4997 - loss: 0.6922

In [15]:
model_list = myprocess.load_models_from_files(num_models=3)
model_list

[<Sequential name=sequential, built=True>,
 <Sequential name=sequential_1, built=True>,
 <Sequential name=sequential_2, built=True>]

In [16]:
for model in model_list:
    test_results = model.evaluate(test_data)
    print('test accuracy : {:.2f}%'. format(test_results[1]*100))

410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9436 - loss: 0.1496
test accuracy : 94.79%
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8759 - loss: 0.3000
test accuracy : 87.73%
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9754 - loss: 0.0725
test accuracy : 97.66%


In [17]:
y_pred = myprocess.majority_vote_ensemble(model_list, test_data, accuracy=0.5)

410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step
410/410 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step


In [18]:
y_pred

array([0, 1, 1, ..., 1, 0, 0])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (410,) + inhomogeneous part.